## Install Libraries

## Import packages and Load Model

In [1]:
%load_ext autoreload
%autoreload 2

import os
import dspy
from dotenv import load_dotenv

load_dotenv('env/var.env')

/Users/rafael/miniconda3/envs/dspy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Models
- we can use OpenAI or HuggingFace models

In [2]:
lm=dspy.OpenAI(api_key=os.getenv("OPENAI_KEY"), model='gpt-3.5-turbo-1106')
#lm = dspy.HFModel(model='mistralai/Mistral-7B-Instruct-v0.1', token=os.getenv("HF_TOKEN"))
dspy.configure(lm=lm)

# DSPy in action:
1. DSPy is an interesting library to test LLMs in different contexts/problems by just changing the `Signature`
2. Signatures in DSPy allow us to ask a LLM to perform different tasks without much prompt engineering, for example:
 - Sentiment Analysis `sentence -> sentiment`
 - Summarization `document -> summary`
 - RAG `context, question -> answer`
3. But it can be even more personalized by bulding our own Signature Classes

## Signatures

### 1. Sentence -> Sentiment (Sentiment Analysis)
- Default

In [3]:
# basic example of classifying sentiment
sentence = "AI has the potential to enhance human capabilities and automate tedious tasks, thereby improving our productivity and quality of life."
classify = dspy.Predict('sentence -> sentiment')
result = classify(sentence=sentence)
print(result.sentiment)

Positive


In [25]:
sentence = "Poorly designed or uncontrolled AI systems pose risks such as job displacement, privacy violations, and even existential threats if advanced AI becomes misaligned with human values."
classify = dspy.Predict('sentence -> sentiment')
result = classify(sentence=sentence)
print(result.sentiment)

Neutral


In [26]:
sentence = "AI can bring existential threats if it becomes misaligned with human values."
classify = dspy.Predict('sentence -> sentiment')
result = classify(sentence=sentence)
print(result.sentiment)

Negative


- Personalized

In [28]:
# an example below of the a custom signature and with a defined output
class ReasonList(dspy.Signature):
    """Classify reason among no_need, too_large, too_small, does_not_look_good"""

    sentence = dspy.InputField()
    reason = dspy.OutputField(desc="A list of values of any one of no_need, too_large, too_small or does_not_look_good", foramt=list[str])

sentence = "I'm returning this item because my sister offered me a similar one"

classify = dspy.Predict(ReasonList)
result = classify(sentence=sentence)
print(result.reason)

sentence = "I'm returning this item because I don't like to see me with it"

classify = dspy.Predict(ReasonList)
result = classify(sentence=sentence)
print(result.reason)

no_need
does_not_look_good


### 2. Document -> Summary (Summarization)

In [39]:
# an example below of the same signature but with different modules
document = """AI technologies hold great promise to enhance and augment human capabilities in a wide range of domains. One of the key benefits of AI is its ability to automate routine and repetitive tasks, freeing up human workers to focus on more complex, creative, and strategic work. AI-powered automation can drive gains in productivity, efficiency, and cost savings across industries.

Additionally, AI systems excel at quickly processing and analyzing large datasets, identifying patterns and insights that may elude human cognition. This data-driven decision making can lead to more informed, evidence-based choices in fields like healthcare, finance, transportation, and scientific research. For example, AI algorithms can assist doctors in early disease detection, optimize logistics and supply chains, and accelerate drug discovery.

Furthermore, AI has transformative potential in enhancing human experiences. AI-powered personal assistants, chatbots, and recommender systems can provide personalized assistance, information, and content tailored to individual needs and preferences. This can improve customer service, education, and quality of life. Advancements in natural language processing, computer vision, and robotic technologies also hold promise for assisting the elderly and disabled, improving accessibility, and extending human physical capabilities.

Lastly, AI may play a vital role in addressing global challenges, such as climate change, resource scarcity, and disease outbreaks. AI systems can help optimize energy grids, model climate patterns, accelerate scientific research, and coordinate disaster response efforts more efficiently than human-only approaches.

Of course, the development and deployment of AI also come with important ethical considerations and potential risks that must be carefully navigated. But overall, the transformative potential of AI to augment and empower humanity is profound and worth continued responsible exploration and investment."""

summarize = dspy.Predict('document -> summary')
response = summarize(document=document)
print(response)

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)
print(response.summary)

Prediction(
    summary='AI technologies have the potential to enhance and augment human capabilities in various domains by automating routine tasks, processing large datasets, and providing personalized assistance. They can also contribute to addressing global challenges. However, ethical considerations and potential risks must be carefully navigated.'
)
AI technologies hold great promise to automate routine tasks, drive productivity and efficiency gains, and enable data-driven decision making across industries. They also have the potential to enhance human experiences, address global challenges, and extend human physical capabilities. However, the development and deployment of AI also come with important ethical considerations and potential risks that must be carefully navigated.


In [38]:
print(lm.inspect_history(n=1))




Given the fields `document`, produce the fields `summary`.

---

Follow the following format.

Document: ${document}
Reasoning: Let's think step by step in order to ${produce the summary}. We ...
Summary: ${summary}

---

Document: AI technologies hold great promise to enhance and augment human capabilities in a wide range of domains. One of the key benefits of AI is its ability to automate routine and repetitive tasks, freeing up human workers to focus on more complex, creative, and strategic work. AI-powered automation can drive gains in productivity, efficiency, and cost savings across industries. Additionally, AI systems excel at quickly processing and analyzing large datasets, identifying patterns and insights that may elude human cognition. This data-driven decision making can lead to more informed, evidence-based choices in fields like healthcare, finance, transportation, and scientific research. For example, AI algorithms can assist doctors in early disease detection, optimi

## 3. Context, Question -> Answer (RAG)

In [48]:
# an example below of the a custom signature using the basic RAG structure.
context = """
Context: Nvidia's stock has nearly tripled so far this year, compared with a rise of about 19% in Microsoft shares, with demand for its top-of-the-line processors outpacing supply.
Tech giants Microsoft, Meta Platforms (META.O), opens new tab and Google-owner Alphabet (GOOGL.O), opens new tab are competing to build out their AI computing capabilities and add the technology to their products and services.
An insatiable appetite for Nvidia's AI processors, viewed as far superior to competitors' offerings, has left them in tight supply, and many investors view Nvidia as the greatest winner to date from surging AI development.
Although in the last few weeks we have been seing NVIDIA reaching the top of the most valued companies in the world, it was surpassed by Apple after the news about their heavy investment in AI.
"""
question = "What is the most valuable company?"

qa = dspy.ChainOfThought('context, question -> answer', temperature=0.7)
response = qa(context=context, question=question)
print(response.answer)

As of the recent developments, Apple has surpassed Nvidia and is currently the most valuable company, following their heavy investment in AI.


In [49]:
print(lm.inspect_history(n=1))




Given the fields `context`, `question`, produce the fields `answer`.

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:

Context: Nvidia's stock has nearly tripled so far this year, compared with a rise of about 19% in Microsoft shares, with demand for its top-of-the-line processors outpacing supply.
Tech giants Microsoft, Meta Platforms (META.O), opens new tab and Google-owner Alphabet (GOOGL.O), opens new tab are competing to build out their AI computing capabilities and add the technology to their products and services.
An insatiable appetite for Nvidia's AI processors, viewed as far superior to competitors' offerings, has left them in tight supply, and many investors view Nvidia as the greatest winner to date from surging AI development.
Although in the last few weeks we have been seing NVIDIA reaching the top of the most valued companies i